# QLoRA 파인튜닝 실습

## 4.2.1 QLoRA 설치 및 설정

In [ ]:
!pip install transformers peft bitsandbytes

## 4.2.2 QLoRA 파인튜닝 예제 - KoAlpaca 데이터셋을 사용한 4비트 파인튜닝

### 모델과 토크나이저 로드 및 양자화 설정

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model

# 4비트 양자화 모델 로드
model_name = "meta-llama/Llama-3.2-1B"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


### 1. QLoRA 설정

In [ ]:
# QLoRA 설정
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)

# QLoRA 모델 생성
model = get_peft_model(model, lora_config)


### 2. KoAlpaca 데이터셋 로드 및 전처리

In [ ]:
from datasets import load_dataset

# KoAlpaca 데이터셋 로드 및 전처리
dataset = load_dataset("beomi/KoAlpaca-v1.1a", split="train")
dataset = dataset.map(lambda x: tokenizer(x['instruction'] + x['output'], padding="max_length", truncation=True, max_length=512, return_tensors="pt"))


### 3. 훈련 설정

In [ ]:
from transformers import Trainer, TrainingArguments

# 훈련 설정
training_args = TrainingArguments(
    output_dir="./qlora_llama_finetune",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_steps=10,
    logging_steps=10,
    logging_dir='./logs'
)


### 4. Trainer 객체 생성 및 파인튜닝 실행

In [ ]:
# Trainer 객체 생성 및 파인튜닝 실행
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset
)
trainer.train()


### 5. 파인튜닝된 모델 저장

In [ ]:
# 파인튜닝된 모델 저장
model.save_pretrained("./qlora_llama_finetuned")


## 4.2.3 파인튜닝 결과 확인

In [ ]:
# 파인튜닝된 모델 로드
model = AutoModelForCausalLM.from_pretrained("./qlora_llama_finetuned")
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 결과 확인
input_text = "KoAlpaca 데이터셋을 이용한 학습 결과는"
inputs = tokenizer(input_text, return_tensors="pt")
output = model.generate(inputs["input_ids"], max_length=50)
print("QLoRA 파인튜닝 응답:", tokenizer.decode(output[0], skip_special_tokens=True))
